In [1]:
#Import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from sklearn.metrics import confusion_matrix, precision_score, recall_score

from keras.preprocessing.image import ImageDataGenerator, array_to_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, SeparableConv2D, Dropout, BatchNormalization

from datetime import datetime
import tensorflow as tf
from google.colab import drive

import os

from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

Using TensorFlow backend.


In [2]:
#Import dataset from google drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%cd drive/My\ Drive/Colab\ Storage\ Real

/content/drive/My Drive/Colab Storage Real


In [ ]:
!ls

ALEXNET_224x224_Model  checkpoint		   test   val
ALEXNET_500x500_Model  checkpoint_alexnet_500x500  train


In [4]:
#Define hyper parameters
hyper_dimension = 227
hyper_epochs = 10
hyper_batch_size = 16
hyper_feature_maps = 32
hyper_channels = 1
hyper_mode = 'grayscale'

In [5]:
#Build model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(96, (11,11), strides=(4,4), activation='relu', input_shape=(hyper_dimension, hyper_dimension, hyper_channels)),
    tf.keras.layers.MaxPooling2D(pool_size = (3,3), strides=(2,2)),
    tf.keras.layers.Conv2D(256, (5,5), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size = (3,3), strides=(2,2)),
    tf.keras.layers.Conv2D(384, (3,3), padding='same', activation='relu'),
    tf.keras.layers.Conv2D(384, (3,3), padding='same', activation='relu'),
    tf.keras.layers.Conv2D(256, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size = (3,3), strides=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 55, 55, 96)        11712     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 384)       885120    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 384)       1327488   
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 256)       8

In [7]:
# Creating training and validation image flows

train_gen = ImageDataGenerator(rescale = 1./255,
                               shear_range = 0.2,
                               zoom_range = 0.2,
                               vertical_flip = True)

val_gen = ImageDataGenerator(rescale = 1./255)

train_set = train_gen.flow_from_directory('train',
                                          target_size = (hyper_dimension,
                                                         hyper_dimension),
                                          batch_size = hyper_batch_size,
                                          class_mode = 'binary',
                                          color_mode = hyper_mode)

val_set = val_gen.flow_from_directory('val',
                                      target_size = (hyper_dimension,
                                                     hyper_dimension),
                                      batch_size = hyper_batch_size,
                                      class_mode = 'binary',
                                      color_mode = hyper_mode)

Found 4192 images belonging to 2 classes.
Found 1057 images belonging to 2 classes.


In [8]:
#Define checkpoint callbacks
checkpoint_path = "checkpoint_alexnet_227x227/cp.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=10 * len(train_set))

In [ ]:
# Compiling the CNN
model.compile(optimizer = tf.keras.optimizers.Adam( learning_rate=0.0001),
                   loss = 'binary_crossentropy',
                   metrics = ['accuracy'])

# Fitting the model

results = model.fit_generator(train_set,
                                   steps_per_epoch = len(train_set),
                                   epochs = hyper_epochs,
                                   validation_data = val_set,
                                   validation_steps = len(val_set), callbacks=[cp_callback])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
262/262 [==============================] - 1630s 6s/step - loss: 0.1124 - accuracy: 0.9573 - val_loss: 0.1466 - val_accuracy: 0.9499
Epoch 2/10
262/262 [==============================] - 120s 460ms/step - loss: 0.1228 - accuracy: 0.9554 - val_loss: 0.1703 - val_accuracy: 0.9272
Epoch 3/10
262/262 [==============================] - 116s 444ms/step - loss: 0.1198 - accuracy: 0.9561 - val_loss: 0.1591 - val_accuracy: 0.9432
Epoch 4/10
262/262 [==============================] - 116s 442ms/step - loss: 0.1212 - accuracy: 0.9525 - val_loss: 0.1399 - val_accuracy: 0.9499
Epoch 5/10
262/262 [==============================] - 116s 443ms/step - loss: 0.1223 - accuracy: 0.9566 - val_loss: 0.1687 - val_accuracy: 0.9385
Epoch 6/10
262/262 [==============================] - 116s 443ms/step - loss: 0.1222 - accuracy: 0.9521 - val_loss: 0.1531 - val_accuracy: 0.9432
Epoch 7/10
262/262 [==============================

In [ ]:
#Save model
model.save('ALEXNET_model.h5')

In [12]:
# Creating test image flow

test_gen = ImageDataGenerator(rescale = 1./255)

test_set = test_gen.flow_from_directory('test',
                                        target_size = (hyper_dimension,
                                                       hyper_dimension),
                                        batch_size = 1,
                                        class_mode = None,
                                        color_mode = hyper_mode,
                                        shuffle=False)

# Making predictions

predictions = model.predict(test_set)
predictions[predictions <= 0.5] = 0
predictions[predictions > 0.5] = 1

Found 624 images belonging to 2 classes.


In [13]:
cm = pd.DataFrame(data=confusion_matrix(test_set.classes, predictions, labels=[0, 1]),
                  index=["Actual Normal", "Actual Pneumonia"],
                  columns=["Predicted Normal", "Predicted Pneumonia"])
print("Precision: ", precision_score(test_set.classes, predictions))
print("Recall: ", recall_score(test_set.classes, predictions))
cm

Precision:  0.8772727272727273
Recall:  0.9897435897435898


,Predicted Normal,Predicted Pneumonia
Actual Normal,180,54
Actual Pneumonia,4,386
